# Doğrusal Regresyon Egzersizleri

50 adet Startup'ın araştırma ve geliştirmeye yönelik harcaması, yönetime yönelik harcaması, pazarlama harcaması, kazandıkları kar miktarı ve kuruldukları lokasyon bilgisi bulunmaktadır. Amaç kar miktarını tahmin etmektir. Bu bir sayısal tahmin problemidir ve bağımlı değişkenimiz "Profit".

Numpy, matplotlib.pyplot, pandas ve seaborn kütüphanelerini çekirdeğe dahil edelim.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Dizinde bulunan veri çerçevemizi startups değişkenine atayalım. startups değişkenini df değişkenine kopyalayarak kullanmaya başlayalım.

In [ ]:
startups = pd.read_csv("../input/startups-50/50_Startups.csv")
df = startups.copy()
df

İlk 5 gözlemini yazdıralım.

In [ ]:
df.head()

Veri çerçevesinin bilgilerini görüntüleyelim.

In [ ]:
df.info()
# Veri setinin bilgilerini görüntülemek için info fonk. kullandık.

Kaç gözlem ve öznitelikten oluştuğunu görüntüleyelim.

In [ ]:
df.shape
#5 öznitelik 50 gözlemden oluşuyor.

Eksik verileri kontrol edelim.

In [ ]:
df.isna().sum()
#Eksik verileri ve hangi verilerden kaç adet eksik olduğunu gördük.
#isna fonksiyonu sorgulama yaptı.
#sum fonksiyonu eksik değer sayısını verdi.
#Eksik değer olmadığını gördük.

Korelasyon matrisi çizdirelim.

In [ ]:
corr = df.corr()
corr
#Korelasyon değeri 1'e yaklaştıkça iki değer arasındaki ilişki o kadar güçlenir.
#En güçlü ve pozitif iki değer R&D Spend ve Profit olduğunu gördük.

Seaborn ile korelasyon matrisinin ısı haritasını çizdirelim.

In [ ]:
corr = df.corr()
sns.heatmap(corr,
           xticklabels=corr.columns.values,
           yticklabels=corr.columns.values);

R&D Spend ve Profit arasındaki korelasyonu daha iyi görebilmek için scatterplot çizdirelim.

In [ ]:
sns.scatterplot(x = "R&D Spend", y = "Profit", data=df, color="aqua");

Sayısal değişkenlerin dağılımını görmek için df üzerinden histogram çizdirelim.

In [ ]:
df.hist(figsize = (13,10))
plt.show()

Veri çerçevesinin temel istatistik değerlerini görüntüleyelim.

In [ ]:
df.describe().T

State'a ait benzersiz değerleri görüntüleyelim.

In [ ]:
df["State"].unique()

get_dummies yardımıyla State'a dair kategorik öznitelik çıkarımlarında bulunalım. Çünkü State'ların birbirine üstünlüğü yok, nominaller. Ordinal değil.

In [ ]:
df_State = pd.get_dummies(df["State"])
#Dummie değişkenine çevirdik kategorik değişkenleri.

In [ ]:
dfDummies = pd.get_dummies(df["State"], prefix="State")
#prefix="State" kullandık çünkü sütun isimlerine ek getirdik

In [ ]:
dfDummies

In [ ]:
df = pd.concat([df,dfDummies], axis=1)

State özniteliğini silip dummy olarak yaratılan State'lardan da birisini hariç tutarak veri çerçevemizi güncelleyelim.

In [ ]:
df = df.drop(["State"], axis=1)
df = df.drop(["State_California"], axis=1)

In [ ]:
df.head()

Veri çerçevemizi bağımlı ve bağımsız değişkenler olmak üzere bölütleyelim.

In [ ]:
A = df.drop("Profit", axis=1)
B = df["Profit"]

Bağımlı ve bağımsız değişkenleri kontrol edelim.

In [ ]:
B
#Bağımlı değişkenlerimiz olan profiti görüntüledik.

In [ ]:
A
#Burada bağımsız değişkenlerimizi görüntüledik

Bu bağımlı ve bağımsız değişkenlerden train ve test olmak üzere 4 parça oluşturalım. Bunu yapmak için train_test_split kullanalım.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.20, random_state=42)

4 parça değişkeni kontrol edelim.

In [ ]:
A_train

In [ ]:
A_test

In [ ]:
B_train

In [ ]:
B_test

LinearRegression'u çekirdeğe dahil edip modeli inşa edelim.

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

Modeli eğitmek için bağımlı bağımsız değişkenlerden oluşturulan eğitim verilerini verelim.

In [ ]:
model = lm.fit(A_train, B_train)

Modele daha önce görmediği bağımlı test değişkenini tahmin ettirelim. Bu tahmin değerlerimizi y_pred değişkenine atayalım.

In [ ]:
B_pred = model.predict(A_test)
B_pred

Tahminleri ve gerçek değerleri bir veri çerçevesinde toplayıp üzerinde göz gezdirelim.

In [ ]:
df = pd.DataFrame({"Gercek Degerler" : B_test, "Tahmin Edilen Degerler" : B_pred, "B_pred ve B_test arasındaki fark:":abs(B_pred-B_test)})
df

sklearn bünyesinde barınan metrics'i çekirdeğe dahil edelim ve MAE, MSE, RMSE değerlerini görüntüleyelim.

In [ ]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(B_test, B_pred)
MAE

In [ ]:
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(B_test, B_pred)
MSE

In [ ]:
import numpy as np
RMSE = np.sqrt(MSE)
RMSE

Modelin R Squared değerini eğitim verileri üzerinden yazdıralım.

In [ ]:
model.score(A_train, B_train)

Dileyenler statsmodel kullanarak hangi özniteliklerin model için %95 güvenilirlikle ne kadar anlamlı olup olmadığına da bakabilir. Modelde bazı feature selection işlemleri yaparak tekrardan eğitip yeni sonuçlar mukayese edilebilir.